![](https://profashion.ru/upload/iblock/bf6/f721319i3l8n3d5tn3tt0fvs207a200m/h_m820.jpg)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import seaborn as sns
import matplotlib.pyplot as plt
import cv2

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
DATA_FOLDER = '/kaggle/input/h-and-m-personalized-fashion-recommendations'

In [ ]:
%%time
articles_df = pd.read_csv(f'{DATA_FOLDER}/articles.csv', dtype={'article_id': str})
customers_df = pd.read_csv(f"{DATA_FOLDER}/customers.csv")
sample_submission_df = pd.read_csv(f"{DATA_FOLDER}/sample_submission.csv", dtype={'article_id': str})

In [ ]:
transactions_train_df = pd.read_csv("/kaggle/input/h-and-m-personalized-fashion-recommendations/transactions_train.csv", dtype={'article_id': str})

In [ ]:
articles_df.head().T

In [ ]:
articles_df.shape

## Amount of unique products in dataset

#### The most interesting things that for each `product_code` several `article_id`s can be exist. So, the `product_code` here represents UPC (Universal Product Code) and `article_id` is a SKU (Stock keeping unit) which is assigned to a product with a different properties of the same product. Let's define how many unique products our dataset contains

In [ ]:
articles_df.article_id.shape[0]==articles_df.shape[0]

In [ ]:
articles_df.product_code.nunique()

#### So that, dataset consist of 47224 unique products

# Articles EDA

In [ ]:
temp = articles_df.groupby('garment_group_name')['garment_group_name'].count()
df = pd.DataFrame({'garment_group_name': temp.index,
                   'amount': temp.values
                  })
df = df.sort_values(['amount'], ascending=False)
plt.figure(figsize = (20,8))
plt.title('Garment Group distribution')
sns.set_color_codes("pastel")
s = sns.barplot(x = 'garment_group_name', y="amount", data=df)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
locs, labels = plt.xticks()
plt.show()

#### There are 5 main categories that appears in dataset and they have following distribution

In [ ]:
temp = articles_df.groupby('index_group_name')['index_group_name'].count()
df = pd.DataFrame({'index_group_name': temp.index,
                   'amount': temp.values
                  })
df = df.sort_values(['amount'], ascending=False)
plt.figure(figsize = (15,6))
plt.title('Main categories distribution')
sns.set_color_codes("pastel")
s = sns.barplot(x = 'index_group_name', y="amount", data=df)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
locs, labels = plt.xticks()
plt.show()

#### Each of this main categories have a leaf categories and they have the following distribution

In [ ]:
temp = articles_df.groupby('section_name')['section_name'].count()
df = pd.DataFrame({'section_name': temp.index,
                   'amount': temp.values
                  })
df = df.sort_values(['amount'], ascending=False)
plt.figure(figsize = (20,8))
plt.title('distribution of sections')
sns.set_color_codes("pastel")
s = sns.barplot(x = 'section_name', y="amount", data=df)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
locs, labels = plt.xticks()
plt.show()

#### So, let's have a look on a distribution of leaf categories separetley grouped by main categories

In [ ]:
_temp = articles_df.groupby('index_group_name')['section_name']
for group_name, _df in _temp:
    temp = _df.value_counts()
    df = pd.DataFrame({group_name: temp.index,
                       'amount': temp.values
                      })
    df = df.sort_values(['amount'], ascending=False)
    plt.figure(figsize = (20,8))
    plt.title(f'Distribution of sections inside {group_name} category')
    sns.set_color_codes("pastel")
    s = sns.barplot(x = group_name, y="amount", data=df)
    s.set_xticklabels(s.get_xticklabels(),rotation=90)
    locs, labels = plt.xticks()
plt.show()

# Clients EDA

In [ ]:
customers_df.head()

In [ ]:
customers_df.shape

In [ ]:
customers_df.info()

In [ ]:
customers_df.Active.unique()

In [ ]:
customers_df.FN.unique()

#### so all `np.nan` values can be replaced with 0 value for example

In [ ]:
plt.figure(figsize=(10,8))
sns.histplot(customers_df.fashion_news_frequency)

#### So, we can see that `None`, `np.nan`, `NONE` can be grouped into one common label `None`

In [ ]:
customers_df.fashion_news_frequency[customers_df.fashion_news_frequency.isin(['NONE',np.nan])]="None"

In [ ]:
plt.figure(figsize=(10,8))
plt.yscale("log")
plt.title("Distribution of newsletter subscription")
sns.histplot(customers_df.fashion_news_frequency)
plt.show()

In [ ]:
plt.figure(figsize=(10,8))
sns.histplot(x='club_member_status', data=customers_df)
plt.title("Distribution of statuses")
plt.show()

In [ ]:
plt.figure(figsize=(20,8))
temp = customers_df.groupby('age')['customer_id'].count()
df = pd.DataFrame({
                    "age": temp.index,
                    "count": temp.values
                   })
s = sns.barplot(x='age', y='count', data=df)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
locs, labels = plt.xticks()
plt.title("Customers' age distribution")
plt.show()

#### As we can see from the Figure above, age has a multimodal distribution

# MORE INSIGHTS WILL BE RELEASED SOON